### Descriptor Feature Selection with RReliefF (Relief for Regression)

### Dataset: MAP Kinase ERK2 Bioactivity data from ChEMBL Database

### Import Libraries and data file

In [1]:
# Base libraries
import pandas as pd
import numpy as np

# rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from rdkit.Chem import Descriptors as des
from rdkit.Chem.Descriptors import qed
from rdkit.Chem import QED

# molvs
from molvs import standardize_smiles

# Data Transformation
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import power_transform

# Feature Selection
import sklearn_relief as sr

# Model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
x=pd.read_csv('erk2.csv')
print(x.shape)
x.head(3)

(23306, 45)


,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL440356,NaN,0.0,243.06,0,1.09,2,O=C1CCNC(=O)c2[nH]c(Br)cc21,IC50,'=',...,MAP kinase ERK2,Homo sapiens,SINGLE PROTEIN,CHEMBL1135814,1.0,Scientific Literature,J. Med. Chem.,2002.0,None,NaN
1,CHEMBL260417,NaN,0.0,373.21,0,3.44,SB-725317,O=C(Nc1n[nH]c2nc(-c3ccc(O)cc3)c(Br)cc12)C1CC1,Inhibition,'=',...,MAP kinase ERK2,Homo sapiens,SINGLE PROTEIN,CHEMBL1961873,16.0,GSK Published Kinase Inhibitor Set,NaN,NaN,None,NaN
2,CHEMBL213451,NaN,0.0,323.33,0,3.62,43,CCNc1nnc2ccc(-c3ocnc3-c3ccc(F)cc3)cn12,IC50,'>',...,MAP kinase ERK2,Homo sapiens,SINGLE PROTEIN,CHEMBL1145312,1.0,Scientific Literature,Bioorg. Med. Chem. Lett.,2006.0,None,NaN


### Data Preprocessing

In [3]:
x1=x[['Molecule ChEMBL ID','Smiles', 'Standard Type', 'Standard Value','Standard Units']]
x1=x1[x1['Standard Units'].str.contains('nM', na=False)]
x1.drop_duplicates(keep='first', inplace=True)
print(x1.shape)
x1.head(3)

(18810, 5)


,Molecule ChEMBL ID,Smiles,Standard Type,Standard Value,Standard Units
0,CHEMBL440356,O=C1CCNC(=O)c2[nH]c(Br)cc21,IC50,539.0,nM
2,CHEMBL213451,CCNc1nnc2ccc(-c3ocnc3-c3ccc(F)cc3)cn12,IC50,10000.0,nM
4,CHEMBL214198,CC(C)c1nnc2ccc(-c3c[nH]nc3-c3cc(F)ccc3F)cn12,IC50,10000.0,nM


In [4]:
x1.isnull().apply(pd.value_counts)

,Molecule ChEMBL ID,Smiles,Standard Type,Standard Value,Standard Units
False,18810.0,18762,18810.0,18806,18810.0
True,NaN,48,NaN,4,NaN


In [5]:
x1.dropna(inplace=True)
print(x1.shape)

(18758, 5)


In [6]:
x1['New Std_value']=x1.groupby('Molecule ChEMBL ID')['Standard Value'].transform('mean')
x1.drop_duplicates('Molecule ChEMBL ID', inplace=True)
x1=x1.drop(['Standard Type', 'Standard Value', 'Standard Units'], axis=1)
x1=x1.sort_values('New Std_value').reset_index(drop=True)
print(x1.shape)
x1.head(3)

(17739, 3)


,Molecule ChEMBL ID,Smiles,New Std_value
0,CHEMBL4868141,Nc1ncc(-c2ccc(NS(=O)(=O)C3CC3)cc2OC2CCCCC2)cc1...,-29600.00000
1,CHEMBL4115001,Nc1ncc([C@@H]2CC[C@@H](O)[C@H](O)C2)nc1-c1ccc(...,0.00431
2,CHEMBL4111166,NC[C@@H](NC(=O)c1ccc(-c2nc([C@@H]3CC[C@@H](O)[...,0.00500


In [7]:
std_smiles=[standardize_smiles(smi) for smi in x1['Smiles']]
std_smiles_df=pd.DataFrame(std_smiles, columns=['Std_Smiles'])

In [8]:
x2=pd.concat([x1[['Molecule ChEMBL ID', 'New Std_value']], std_smiles_df], axis=1)
print(x2.shape)
x2.head(3)

(17739, 3)


,Molecule ChEMBL ID,New Std_value,Std_Smiles
0,CHEMBL4868141,-29600.00000,Nc1ncc(-c2ccc(NS(=O)(=O)C3CC3)cc2OC2CCCCC2)cc1...
1,CHEMBL4115001,0.00431,Nc1ncc([C@@H]2CC[C@@H](O)[C@H](O)C2)nc1-c1ccc(...
2,CHEMBL4111166,0.00500,NC[C@@H](NC(=O)c1ccc(-c2nc([C@@H]3CC[C@@H](O)[...


### Label Actives and Inactives

In [9]:
x2['Label']=x2['New Std_value'].apply(lambda x: 1 if x <=10000 else 0)
x2.head(3)

,Molecule ChEMBL ID,New Std_value,Std_Smiles,Label
0,CHEMBL4868141,-29600.00000,Nc1ncc(-c2ccc(NS(=O)(=O)C3CC3)cc2OC2CCCCC2)cc1...,1
1,CHEMBL4115001,0.00431,Nc1ncc([C@@H]2CC[C@@H](O)[C@H](O)C2)nc1-c1ccc(...,1
2,CHEMBL4111166,0.00500,NC[C@@H](NC(=O)c1ccc(-c2nc([C@@H]3CC[C@@H](O)[...,1


In [10]:
x2['Label'].value_counts()

0    10534
1     7205
Name: Label, dtype: int64

In [11]:
x2[['Std_Smiles','New Std_value']].to_csv('erk2_regression.smi', sep='\t', index=None, header=None)

### RDKit Descriptors

In [12]:
supplier=Chem.SmilesMolSupplier('erk2_regression.smi', delimiter='\t', titleLine=False)

In [13]:
qed, ExactMolWt, MolLogP, TPSA, numHA, numHD, Std_Value = [],[],[],[],[],[],[] 

for i,mol in enumerate(supplier):
    molH=Chem.AddHs(mol)
    qed.append(des.qed(molH))
    ExactMolWt.append(des.ExactMolWt(molH))
    MolLogP.append(des.MolLogP(molH))
    TPSA.append(des.TPSA(molH))
    numHA.append(des.NumHAcceptors(molH))
    numHD.append(des.NumHDonors(molH))
    Std_Value.append(mol.GetProp('_Name'))

In [14]:
descriptors=list(zip(qed, ExactMolWt, MolLogP, TPSA, numHA, numHD, Std_Value))
desc_df=pd.DataFrame(descriptors, columns=['qed', 'ExactMolWt', 'MolLogP', 'TPSA', 'numHA', 'numHD', 'Std_Value'])
desc_df.head()

,qed,ExactMolWt,MolLogP,TPSA,numHA,numHD,Std_Value
0,0.397743,532.214427,4.8993,123.41,6,3,-29600.0
1,0.270421,610.088860,3.0613,141.59,7,5,0.00431
2,0.312455,611.100508,4.0049,127.15,6,4,0.005
3,0.270421,610.088860,3.0613,141.59,7,5,0.0055
4,0.381566,579.094279,3.9324,116.15,6,3,0.00612


### Scale data

In [15]:
scaler=MinMaxScaler()

In [16]:
scaledData=power_transform(desc_df, method='yeo-johnson')
scaledData=scaler.fit_transform(scaledData)
print(scaledData.shape)
scaledData

(17739, 7)


array([[0.36294761, 0.44200627, 0.471274  , ..., 0.29132204, 0.30335564,
        0.        ],
       [0.235572  , 0.47427127, 0.37697699, ..., 0.31987807, 0.39631998,
        0.46405025],
       [0.27675052, 0.47466669, 0.42383281, ..., 0.29132204, 0.35427354,
        0.46405025],
       ...,
       [0.81397936, 0.26098163, 0.30936194, ..., 0.13907291, 0.14892822,
        0.61418504],
       [0.81397936, 0.26098163, 0.30936194, ..., 0.13907291, 0.14892822,
        0.61418504],
       [0.72939581, 0.36289692, 0.34293624, ..., 0.26003151, 0.23858206,
        1.        ]])

In [17]:
scaledData_df=pd.DataFrame(scaledData, columns=[['qed', 'ExactMolWt', 'MolLogP', 'TPSA', 'numHA', 'numHD', 'Std_Value']])
Features=scaledData_df[['qed', 'ExactMolWt', 'MolLogP', 'TPSA', 'numHA', 'numHD']]
Features.head(3)

,qed,ExactMolWt,MolLogP,TPSA,numHA,numHD
0,0.362948,0.442006,0.471274,0.269596,0.291322,0.303356
1,0.235572,0.474271,0.376977,0.288866,0.319878,0.396320
2,0.276751,0.474667,0.423833,0.273680,0.291322,0.354274


### Apply RRelief algorithim and deduce feature importance

In [18]:
X=scaledData[:,0:6]
y=scaledData[:,6:7]
print(X.shape)
print(y.shape)

(17739, 6)
(17739, 1)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [20]:
RR=sr.RReliefF(n_features=6)
feature_importance=RR.fit_transform(X_train,y_train.ravel())

In [21]:
feature_importance

array([[0.44847002, 0.28354096, 0.24823155, 0.23858206, 0.39383239,
        0.4786291 ],
       [0.33834123, 0.18507858, 0.54182413, 0.23858206, 0.26003151,
        0.2917241 ],
       [0.35492335, 0.22425825, 0.66658793, 0.30335564, 0.22521192,
        0.34950285],
       ...,
       [0.39198836, 0.1463276 , 0.73896931, 0.        , 0.22521192,
        0.2979378 ],
       [0.32192046, 0.26736139, 0.44866298, 0.35427354, 0.29132204,
        0.35987208],
       [0.39074898, 0.23887791, 0.38717001, 0.23858206, 0.29132204,
        0.37888239]])

In [22]:
feature_importance_df=pd.DataFrame(feature_importance, columns=['1','2','3','4','5','6'])
feature_importance_df.head(3)

,1,2,3,4,5,6
0,0.448470,0.283541,0.248232,0.238582,0.393832,0.478629
1,0.338341,0.185079,0.541824,0.238582,0.260032,0.291724
2,0.354923,0.224258,0.666588,0.303356,0.225212,0.349503


In [23]:
Xtrain_features=pd.DataFrame(X_train, columns=[['qed', 'ExactMolWt', 'MolLogP', 'TPSA', 'numHA', 'numHD']])
Xtrain_features.head(3)

,qed,ExactMolWt,MolLogP,TPSA,numHA,numHD
0,0.248232,0.478629,0.448470,0.283541,0.393832,0.238582
1,0.541824,0.291724,0.338341,0.185079,0.260032,0.238582
2,0.666588,0.349503,0.354923,0.224258,0.225212,0.303356


Feature importance from most to least important: 
1-->MolLogP, 2-->TPSA, 3-->qed, 4-->numHD, 5-->numHA, 6-->ExactMolWt